<a href="https://colab.research.google.com/github/NonthawatNK/Neural_Network-for-led-display/blob/main/Led%20display%20domain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#import

In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import KFold
from keras.utils import plot_model
from keras.optimizers import RMSprop
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

#Path file


```
fileName : ต้นทางของไฟล์ที่เก็บ raw data
Y_pathname : มีแค่ desing ที่ถูกเปลี่ยรูปไปแล้ว
```



In [ ]:
fileName = "/content/drive/MyDrive/Data/datamining/seed5_1000exsample.txt"
Y_pathname = "/content/drive/MyDrive/Data/datamining/Ydummy.csv"

#ดูข้อมูล

In [ ]:
df = pd.read_csv(fileName,",",)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Att_1   1000 non-null   int64
 1   Att_2   1000 non-null   int64
 2   Att_3   1000 non-null   int64
 3   Att_4   1000 non-null   int64
 4   Att_5   1000 non-null   int64
 5   Att_6   1000 non-null   int64
 6   Att_7   1000 non-null   int64
 7   Att_8   1000 non-null   int64
dtypes: int64(8)
memory usage: 62.6 KB


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.head(10)

,Att_1,Att_2,Att_3,Att_4,Att_5,Att_6,Att_7,Att_8
0,1,1,1,1,0,1,1,9
1,1,0,1,0,0,1,0,7
2,0,0,1,0,0,1,0,1
3,1,1,1,1,0,1,1,9
4,0,1,1,1,0,1,0,4
5,1,1,0,1,0,1,1,5
6,1,1,1,1,0,1,1,9
7,0,0,1,0,0,1,0,1
8,1,1,0,1,0,1,1,5
9,1,1,1,0,1,1,1,0


# เขียนใส่ใน Array เพื่อนำไปใช้ในการแบ่ง Fold

In [ ]:
with open(Y_pathname, 'r') as file:
    data_Y = []
    for line in file:
        row = line.strip().split(',')
        data_Y.append(row)
        
with open(fileName, 'r') as file:
    data = []
    for line in file:
        row = line.strip().split(',')
        data.append(row)

#drop header ออก


In [ ]:
data.pop(0)
data[0]
data_Y.pop(0)
data_Y[0]

# Define the model


In [ ]:
model = Sequential()
model.add(Dense( 7,input_dim=7, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

# Compile the model


In [ ]:
lr = 0.01
optimizer = RMSprop(lr=lr)
model.compile(loss='mean_squared_error', optimizer=optimizer,metrics=['accuracy']) 

# Load the data into a numpy array


In [ ]:
data = np.array(data)
data_Y = np.array(data_Y)

# Split the data into input features (X) and target variable (y)


In [ ]:
X = data[:, :-1].astype(int)
Y = data_Y.astype(int)

#Fold

In [ ]:
# Define the number of folds
n_folds = 10
# Define the K-Fold cross-validator
kfold = KFold(n_folds, shuffle=True,random_state= 5)

#Train and Score

In [ ]:
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# Loop through each fold
for fold_no, (train_index, test_index) in enumerate(kfold.split(X)):
    # Split the data into training and testing sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    # print(f"  Train x: shape={X_train.shape}")
    # print(f"  Test x: shape={X_test.shape}")
    # print(f"  Train y: shape={y_train.shape}")
    # print(f"  Test y: shape={y_test.shape}")

    # Fit the model on the training data
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

    # Generate generalization metrics
    scores = model.evaluate(X_test, y_test, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

In [ ]:
# Increase fold number
fold_no = fold_no + 1
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')